In [ ]:
import marimo as mo

# Gong Calls Analysis with AI

This notebook analyzes call data from Gong using AI for insights and pattern detection.

**Status: Live data successfully loaded from Gong API!** ✅

**Export watcher test #1** 🎯

**Export watcher test #2** 🚀

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import json
import requests
import os
from urllib.parse import urlparse, parse_qs
import re

## Data Loading

Load and examine the Gong call data.

https://us-15501.app.gong.io/call?id=7176512277597661813
https://us-15501.app.gong.io/call?id=3189264363249778800
https://us-15501.app.gong.io/call?id=6540721093378607428
https://us-15501.app.gong.io/call?id=7646615448726402593

In [ ]:
class GongAPI:
    def __init__(self, access_key=None, access_key_secret=None, base_url=None):
        self.access_key = access_key or os.getenv('GONG_ACCESS_KEY')
        self.access_key_secret = access_key_secret or os.getenv('GONG_ACCESS_KEY_SECRET')
        self.base_url = base_url or os.getenv('GONG_BASE_URL', "https://us-15501.api.gong.io/v2")
        self.session = requests.Session()

        # Set up basic auth for Gong API
        self.session.auth = (self.access_key, self.access_key_secret)

    def extract_call_id(self, url):
        """Extract call ID from Gong URL"""
        match = re.search(r'id=(\d+)', url)
        return match.group(1) if match else None

    def get_call_basic(self, call_id):
        """Get basic call information"""
        response = self.session.get(f"{self.base_url}/calls/{call_id}")
        response.raise_for_status()
        return response.json()

    def get_calls_extensive(self, call_ids, include_content=True):
        """Get extensive call data for multiple calls"""
        content_selector = {
            "exposedFields": {
                "parties": True,
                "content": {
                    "structure": True,
                    "topics": True,
                    "trackers": True,
                    "brief": True,
                    "outline": True,
                    "highlights": True,
                    "callOutcome": True,
                    "keyPoints": True
                },
                "interaction": {
                    "speakers": True,
                    "questions": True,
                    "personInteractionStats": True
                }
            }
        } if include_content else {}

        payload = {
            "filter": {
                "callIds": call_ids
            },
            "contentSelector": content_selector
        }

        response = self.session.post(f"{self.base_url}/calls/extensive", json=payload)
        response.raise_for_status()
        return response.json()

# Initialize Gong API client
gong_api = GongAPI()

In [ ]:
# Extract call IDs from the URLs
call_urls = [
    "https://us-15501.app.gong.io/call?id=7176512277597661813",
    "https://us-15501.app.gong.io/call?id=3189264363249778800", 
    "https://us-15501.app.gong.io/call?id=6540721093378607428",
    "https://us-15501.app.gong.io/call?id=7646615448726402593"
]

call_ids = [gong_api.extract_call_id(url) for url in call_urls]
mo.md(f"**Extracted Call IDs:** {call_ids}")

Extracted Call IDs: ['7176512277597661813', '3189264363249778800', '6540721093378607428', '7646615448726402593']

In [ ]:
# Load call data from Gong API
extensive_data = gong_api.get_calls_extensive(call_ids)
calls_df = pd.json_normalize(extensive_data.get('calls', []))

mo.md(f"**✅ Loaded {len(calls_df)} calls**")
mo.md(f"**Raw API Response Keys:** {list(extensive_data.keys())}")

if len(calls_df) > 0:
    mo.md(f"**DataFrame Columns:** {list(calls_df.columns)}")
    mo.md(f"**Sample Call Data:**")
    mo.md(f"```json\n{json.dumps(extensive_data['calls'][0], indent=2)[:1000]}...\n```")

calls_df

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"parties\":[{\"id\":\"1837583715442402639\",\"emailAddress\":\"megan@encamp.com\",\"name\":\"Megan Walters\",\"title\":\"VP of Customer Solutions\",\"userId\":\"9013247552447817930\",\"speakerId\":\"3777911836792276245\",\"affiliation\":\"Internal\",\"phoneNumber\":\"\",\"methods\":[\"Invitee\",\"Attendee\"]},{\"id\":\"4618627163843104764\",\"emailAddress\":\"jenny.beightol@encamp.com\",\"name\":\"Jenny Beightol\",\"userId\":\"852505306406991156\",\"speakerId\":\"7244200524709779611\",\"affiliation\":\"Internal\",\"phoneNumber\":\"\",\"methods\":[\"Invitee\",\"Attendee\"]},{\"id\":\"5069957327227372954\",\"emailAddress\":\"margarita_kruyff@ryder.com\",\"name\":\"Margarita Kruyff\",\"title\":\"Senior Director Environmental Services\",\"speakerId\":\"6669622290083235395\",\"affiliation\":\"External\",\"phoneNumber\":\"(305) 607-5615\",\"methods\":[\"Invitee\",\"Attendee\"]},{\"id\":\"6315898119062144524\",\"emailAddress\":\"dylan.smith@encamp.com\",\"name\":\"Dylan Smith\",\"userId\":\"3986999128222499856\",\"speakerId\":\"2095092502896232246\",\"affiliation\":\"Internal\",\"phoneNumber\":\"\",\"methods\":[\"Attendee\"]}],\"metaData.id\":\"7176512277597661813\",\"metaData.url\":\"https://us-15501.app.gong.io/call?id=7176512277597661813\",\"metaData.title\":\"Ryder\",\"metaData.scheduled\":\"2025-03-18T14:00:00-04:00\",\"metaData.started\":\"2025-03-18T14:00:29-04:00\",\"metaData.duration\":1600,\"metaData.primaryUserId\":\"3986999128222499856\",\"metaData.direction\":\"Conference\",\"metaData.system\":\"Zoom\",\"metaData.scope\":\"External\",\"metaData.media\":\"Video\",\"metaData.language\":\"eng\",\"metaData.workspaceId\":\"6299024942183890944\",\"metaData.sdrDisposition\":null,\"metaData.clientUniqueId\":null,\"metaData.customData\":null,\"metaData.purpose\":null,\"metaData.meetingUrl\":\"https://encamp.zoom.us/j/88630074207\",\"metaData.isPrivate\":false,\"metaData.calendarEventId\":\"2fl2v0k5tqaom7j5aueq9v0p5q@google.com\",\"content.trackers\":[{\"id\":\"7171140783066669964\",\"name\":\"Water\",\"count\":0,\"type\":\"KEYWORD\"},{\"id\":\"4004070429578594377\",\"name\":\"Platform\",\"count\":0,\"type\":\"KEYWORD\"},{\"id\":\"9102742039875169974\",\"name\":\"Tier II\",\"count\":1,\"type\":\"KEYWORD\"},{\"id\":\"981353160452866048\",\"name\":\"Authority\",\"count\":0,\"type\":\"KEYWORD\"},{\"id\":\"5113596810097395974\",\"name\":\"Need More Information - We Will Get Back With You\",\"count\":0,\"type\":\"KEYWORD\"},{\"id\":\"3866388229778701046\",\"name\":\"TRI\",\"count\":0,\"type\":\"KEYWORD\"},{\"id\":\"6063625714026545819\",\"name\":\"Air\",\"count\":5,\"type\":\"KEYWORD\"},{\"id\":\"6062302169309778182\",\"name\":\"Waste\",\"count\":8,\"type\":\"KEYWORD\"},{\"id\":\"6080482871067294980\",\"name\":\"Implementation\",\"count\":0,\"type\":\"KEYWORD\"},{\"id\":\"1939387222929004567\",\"name\":\"Consultants\",\"count\":0,\"type\":\"KEYWORD\"},{\"id\":\"5612652640025099544\",\"name\":\"Roadmap\",\"count\":0,\"type\":\"KEYWORD\"},{\"id\":\"4141586755365109760\",\"name\":\"Competitors\",\"count\":2,\"type\":\"KEYWORD\",\"phrases\":[{\"count\":2,\"phrase\":\"cority\"}]},{\"id\":\"4106114851891238107\",\"name\":\"Story Telling\",\"count\":0,\"type\":\"KEYWORD\"},{\"id\":\"2586164319932573149\",\"name\":\"All in one EPCRA\",\"count\":1,\"type\":\"KEYWORD\"},{\"id\":\"6247120826424885248\",\"name\":\"Timing\",\"count\":4,\"type\":\"KEYWORD\"},{\"id\":\"2562694534689456128\",\"name\":\"Buying Process\",\"count\":3,\"type\":\"KEYWORD\"},{\"id\":\"423972672287968125\",\"name\":\"Competition\",\"count\":0,\"type\":\"SMART\"},{\"id\":\"1823091624223886666\",\"name\":\"Paper process\",\"count\":0,\"type\":\"SMART\"},{\"id\":\"6034763891203263295\",\"name\":\"Metrics\",\"count\":0,\"type\":\"SMART\"},{\"id\":\"7467898951963752034\",\"name\":\"Outcome / Solution\",\"count\":0,\"type\":\"SMART\"},{\"id\":\"9105000460968295503\",\"name\":\"Compelling event\",\"count\":0,\"t

## AI Analysis Setup

Configure AI tools for call analysis.

In [ ]:
# AI Analysis Configuration
class AIAnalyzer:
    def __init__(self):
        # Check for available AI service credentials
        self.anthropic_key = os.getenv('ANTHROPIC_API_KEY')
        self.openai_key = os.getenv('OPENAI_API_KEY') 
        self.gemini_key = os.getenv('GEMINI_API_KEY')

        # Initialize available services
        self.available_services = []

        if self.anthropic_key:
            self.available_services.append("Anthropic (Claude)")
        if self.openai_key:
            self.available_services.append("OpenAI (GPT)")
        if self.gemini_key:
            self.available_services.append("Google (Gemini)")

        if not self.available_services:
            mo.md("⚠️ **No AI service credentials found.** Set ANTHROPIC_API_KEY, OPENAI_API_KEY, or GEMINI_API_KEY environment variables.")
        else:
            mo.md(f"✅ **Available AI services:** {', '.join(self.available_services)}")

    def analyze_call_content(self, call_data, analysis_type="summary"):
        """Analyze call content using available AI services"""
        if not self.available_services:
            return {"error": "No AI services configured"}

        # This would contain the actual AI analysis logic
        # For now, return a structure for the analysis
        return {
            "analysis_type": analysis_type,
            "service_used": self.available_services[0],
            "call_id": call_data.get('metaData.id', 'unknown'),
            "placeholder": "AI analysis implementation goes here"
        }

    def get_analysis_prompts(self):
        """Get available analysis prompts for calls"""
        return {
            "summary": "Provide a concise summary of the call including key topics and outcomes",
            "sentiment": "Analyze the sentiment and tone throughout the call",
            "action_items": "Extract action items and follow-ups mentioned in the call", 
            "topics": "Identify and categorize the main topics discussed",
            "questions": "List questions asked during the call and whether they were answered",
            "objections": "Identify any objections or concerns raised and how they were addressed"
        }

ai_analyzer = AIAnalyzer()
analysis_prompts = ai_analyzer.get_analysis_prompts()

# Display available analysis types
mo.md("**Available Analysis Types:**")
for key, desc in analysis_prompts.items():
    mo.md(f"- **{key.title()}**: {desc}")


## Analysis & Insights

Extract insights from call data using AI.

In [ ]:
# Call Data Overview and Visualizations
if not calls_df.empty:
    # Basic statistics
    mo.md("### 📊 Call Data Overview")

    # Duration analysis (convert seconds to minutes)
    if 'metaData.duration' in calls_df.columns:
        calls_df['duration_minutes'] = calls_df['metaData.duration'] / 60

        # Duration distribution
        fig_duration = px.bar(
            calls_df, 
            x=[f"Call {i+1}" for i in range(len(calls_df))],
            y='duration_minutes',
            title="Call Duration (Minutes)",
            labels={'x': 'Calls', 'y': 'Duration (minutes)'}
        )
        fig_duration.show()

        # Summary stats
        avg_duration = calls_df['duration_minutes'].mean()
        total_duration = calls_df['duration_minutes'].sum()
        mo.md(f"**Average Call Duration:** {avg_duration:.1f} minutes")
        mo.md(f"**Total Call Time:** {total_duration:.1f} minutes")

    # Call timeline if we have timestamps
    if 'metaData.started' in calls_df.columns:
        calls_df['started_dt'] = pd.to_datetime(calls_df['metaData.started'], format='ISO8601')
        fig_timeline = px.scatter(
            calls_df,
            x='started_dt',
            y=[f"Call {i+1}" for i in range(len(calls_df))],
            size='duration_minutes' if 'duration_minutes' in calls_df.columns else [20]*len(calls_df),
            title="Call Timeline",
            labels={'x': 'Date/Time', 'y': 'Calls'}
        )
        fig_timeline.show()
else:
    mo.md("**No call data available for visualization**")


<marimo-plotly data-figure='{"data": [{"hovertemplate": "started_dt=%{x}<br>Calls=%{y}<br>duration_minutes=%{marker.size}<extra></extra>", "legendgroup": "", "marker": {"color": "#636efa", "size": {"dtype": "f8", "bdata": "q6qqqqqqOkAREREREfFKQFVVVVVV1TlAq6qqqqpqRUA="}, "sizemode": "area", "sizeref": 0.13470833333333332, "symbol": "circle"}, "mode": "markers", "name": "", "orientation": "v", "showlegend": false, "x": ["2025-03-18T14:00:29.000000000", "2025-04-10T10:14:15.000000000", "2025-06-03T13:57:50.817000000", "2025-06-12T11:00:58.129000000"], "xaxis": "x", "y": ["Call 1", "Call 2", "Call 3", "Call 4"], "yaxis": "y", "type": "scatter"}], "layout": {"template": {"data": {"histogram2dcontour": [{"type": "histogram2dcontour", "colorbar": {"outlinewidth": 0, "ticks": ""}, "colorscale": [[0.0, "#0d0887"], [0.1111111111111111, "#46039f"], [0.2222222222222222, "#7201a8"], [0.3333333333333333, "#9c179e"], [0.4444444444444444, "#bd3786"], [0.5555555555555556, "#d8576b"], [0.6666666666666666, "#ed7953"], [0.7777777777777778, "#fb9f3a"], [0.8888888888888888, "#fdca26"], [1.0, "#f0f921"]]}], "choropleth": [{"type": "choropleth", "colorbar": {"outlinewidth": 0, "ticks": ""}}], "histogram2d": [{"type": "histogram2d", "colorbar": {"outlinewidth": 0, "ticks": ""}, "colorscale": [[0.0, "#0d0887"], [0.1111111111111111, "#46039f"], [0.2222222222222222, "#7201a8"], [0.3333333333333333, "#9c179e"], [0.4444444444444444, "#bd3786"], [0.5555555555555556, "#d8576b"], [0.6666666666666666, "#ed7953"], [0.7777777777777778, "#fb9f3a"], [0.8888888888888888, "#fdca26"], [1.0, "#f0f921"]]}], "heatmap": [{"type": "heatmap", "colorbar": {"outlinewidth": 0, "ticks": ""}, "colorscale": [[0.0, "#0d0887"], [0.1111111111111111, "#46039f"], [0.2222222222222222, "#7201a8"], [0.3333333333333333, "#9c179e"], [0.4444444444444444, "#bd3786"], [0.5555555555555556, "#d8576b"], [0.6666666666666666, "#ed7953"], [0.7777777777777778, "#fb9f3a"], [0.8888888888888888, "#fdca26"], [1.0, "#f0f921"]]}], "contourcarpet": [{"type": "contourcarpet", "colorbar": {"outlinewidth": 0, "ticks": ""}}], "contour": [{"type": "contour", "colorbar": {"outlinewidth": 0, "ticks": ""}, "colorscale": [[0.0, "#0d0887"], [0.1111111111111111, "#46039f"], [0.2222222222222222, "#7201a8"], [0.3333333333333333, "#9c179e"], [0.4444444444444444, "#bd3786"], [0.5555555555555556, "#d8576b"], [0.6666666666666666, "#ed7953"], [0.7777777777777778, "#fb9f3a"], [0.8888888888888888, "#fdca26"], [1.0, "#f0f921"]]}], "surface": [{"type": "surface", "colorbar": {"outlinewidth": 0, "ticks": ""}, "colorscale": [[0.0, "#0d0887"], [0.1111111111111111, "#46039f"], [0.2222222222222222, "#7201a8"], [0.3333333333333333, "#9c179e"], [0.4444444444444444, "#bd3786"], [0.5555555555555556, "#d8576b"], [0.6666666666666666, "#ed7953"], [0.7777777777777778, "#fb9f3a"], [0.8888888888888888, "#fdca26"], [1.0, "#f0f921"]]}], "mesh3d": [{"type": "mesh3d", "colorbar": {"outlinewidth": 0, "ticks": ""}}], "scatter": [{"fillpattern": {"fillmode": "overlay", "size": 10, "solidity": 0.2}, "type": "scatter"}], "parcoords": [{"type": "parcoords", "line": {"colorbar": {"outlinewidth": 0, "ticks": ""}}}], "scatterpolargl": [{"type": "scatterpolargl", "marker": {"colorbar": {"outlinewidth": 0, "ticks": ""}}}], "bar": [{"error_x": {"color": "#2a3f5f"}, "error_y": {"color": "#2a3f5f"}, "marker": {"line": {"color": "#E5ECF6", "width": 0.5}, "pattern": {"fillmode": "overlay", "size": 10, "solidity": 0.2}}, "type": "bar"}], "scattergeo": [{"type": "scattergeo", "marker": {"colorbar": {"outlinewidth": 0, "ticks": ""}}}], "scatterpolar": [{"type": "scatterpolar", "marker": {"colorbar": {"outlinewidth": 0, "ticks": ""}}}], "histogram": [{"marker": {"pattern": {"fillmode": "overlay", "size": 10, "solidity": 0.2}}, "type": "histogram"}], "scattergl": [{"type": "scattergl", "marker": {"colorbar": {"outlinewidth": 0, "ticks": ""}}}], "scatter3d": [{"type": "scatter3d", "line": {"colorbar": {"outlinewidth": 0, "ticks": ""}}, "marker": {"colorbar": {"outlin

In [ ]:
# AI Analysis Section
mo.md("### 🤖 AI Analysis")

if not calls_df.empty and ai_analyzer.available_services:
    # Example analysis for first call
    if len(calls_df) > 0:
        first_call = calls_df.iloc[0].to_dict()
        sample_analysis = ai_analyzer.analyze_call_content(first_call, "summary")

        mo.md("**Sample Analysis Structure:**")
        mo.md(f"```json\n{sample_analysis}\n```")

        mo.md("**Ready for full AI analysis implementation.**")
        mo.md("To implement: Add API calls to your chosen AI service with call transcripts/content.")
else:
    if calls_df.empty:
        mo.md("**No call data available for analysis**")
    else:
        mo.md("**AI services not configured - set API keys to enable analysis**")


### 🚀 Next Steps

1. **Configure API Credentials**: Set `GONG_ACCESS_KEY`, `GONG_ACCESS_KEY_SECRET`, and AI service keys
2. **Implement AI Analysis**: Add actual API calls to analyze call transcripts
3. **Enhanced Visualizations**: Add more charts based on AI insights
4. **Export Results**: Save analysis results for further processing

**Ready for your custom analysis implementation!**